In [ ]:
import os
import re
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import random
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn import Linear, ReLU, Dropout, Conv2d, MaxPool2d
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.optim import AdamW

import gurobipy as gb
from gurobipy import GRB
import time

### Check if GPU available

In [ ]:
# set CUDA_VISIBLE_DEVICES=0
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
train_test_dir = os.path.join(os.getcwd(), "dataGeneration/preprocessed_data_test")

X_test = np.load(os.path.join(train_test_dir, "X_test.npy"))
y_test = np.load(os.path.join(train_test_dir, "y_test.npy"))
index_test = np.load(os.path.join(train_test_dir, "indices_test.npy")).astype("int64")

solTime_test = np.load(os.path.join(train_test_dir, "solTime_test.npy"))
objVal_test = np.load(os.path.join(train_test_dir, "objVal_test.npy"))
schedule_test = np.load(os.path.join(train_test_dir, "schedule_test.npy")).astype("int32")
model_test = np.load(os.path.join(train_test_dir, "model_test.npy")).astype("int32")


In [ ]:
X_test = np.transpose(X_test, (0,1,3,2))

print(X_test.shape)
print(X_test.dtype)
print(y_test.shape)
print(y_test.dtype)
print(index_test.shape)
print(index_test.dtype)

### Building the CNN network

In [ ]:
in_channels = X_test.shape[2]
col = X_test.shape[3]

out_channels = y_test.shape[-1]
nbScen = X_test.shape[1]

print(in_channels, col, out_channels)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.hidden_ch = 64
        self.dp = 0.1

        self.feature_extractor = nn.Sequential(
            nn.Conv1d(in_channels, self.hidden_ch, 11, stride=1, padding=0),
            nn.ReLU(),
            nn.Dropout(self.dp),
            nn.MaxPool1d(5, stride=1, padding=0),

            nn.Conv1d(self.hidden_ch, self.hidden_ch*2, 7, stride=1, padding=0),
            nn.ReLU(),
            nn.Dropout(self.dp),
            nn.MaxPool1d(5, stride=1, padding=0),

            nn.Conv1d(self.hidden_ch*2, self.hidden_ch, 3, stride=1, padding=0),
            nn.ReLU(),
            nn.Dropout(self.dp),
            nn.MaxPool1d(5, stride=1, padding=0),
        )
        
        n_channels = self.feature_extractor(torch.zeros(1, in_channels, col)).size(-1)

        self.classifier = nn.Sequential(
            nn.MaxPool1d(n_channels), # GAP
            nn.Flatten(),
            nn.Linear(self.hidden_ch, self.hidden_ch*2),
            nn.ReLU(),
            nn.Dropout(self.dp),
            nn.Linear(self.hidden_ch*2, self.hidden_ch*2),
            nn.ReLU(),
            nn.Dropout(self.dp),
            nn.Linear(self.hidden_ch*2, out_channels),
            nn.Sigmoid())


    def forward(self, x):
        features = self.feature_extractor(x)
        out = self.classifier(features)
        return out
        

### Create Dataset and DataLoader

In [ ]:
config = {
        'batch_size' : 8, # Num samples to average over for gradient updates
        'EPOCHS' : 500, # Num times to iterate over the entire dataset
        'LEARNING_RATE' : 5e-4, # Learning rate for the optimizer
        'WEIGHT_DECAY' : 1e-4, # Weight decay parameter for the Adam optimizer
    }

In [ ]:
class coordinationDataset(TensorDataset):
    def __init__(self, X, y):
        super(coordinationDataset, self).__init__()
        self.X = X
        self.y = y
        
    def __getitem__(self, index):
        X = self.X[index]
        y = self.y[index]
        
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.round(torch.tensor(y, dtype=torch.float32))

        return X_tensor, y_tensor
    
    def __len__(self):
        return len(self.X)

In [ ]:
test_dataset = coordinationDataset(X_test, y_test)

print(test_dataset.X[0].shape)

In [ ]:
net = NeuralNetwork()

batch_size = config['batch_size']
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

optimizer = optim.Adam(net.parameters(), lr=config["LEARNING_RATE"])
total_steps = len(test_loader)


#### Define custom loss function

In [ ]:
def asymmetric_loss(predict, target, gamma_neg=0.3, gamma_pos=0, clip=0.0, eps=1e-8, disable_torch_grad_focal_loss=True):

    """"
    Parameters
    ----------
    x: input logits
    y: targets (multi-label binarized vector)
    """

    # Calculating Probabilities
    x_sigmoid = predict
    xs_pos = x_sigmoid
    xs_neg = 1 - x_sigmoid

    # Asymmetric Clipping
    if clip is not None and clip > 0:
        xs_neg = (xs_neg + clip).clamp(max=1)

    # Basic CE calculation
    los_pos = target * torch.log(xs_pos.clamp(min=eps))
    los_neg = (1 - target) * torch.log(xs_neg.clamp(min=eps))
    loss = los_pos + los_neg

    # Asymmetric Focusing
    if gamma_neg > 0 or gamma_pos > 0:
        if disable_torch_grad_focal_loss:
            torch.set_grad_enabled(False)
        pt0 = xs_pos * target
        pt1 = xs_neg * (1 - target)  # pt = p if t > 0 else 1-p
        pt = pt0 + pt1
        one_sided_gamma = gamma_pos * target + gamma_neg * (1 - target)
        one_sided_w = torch.pow(1 - pt, one_sided_gamma)
        if disable_torch_grad_focal_loss:
            torch.set_grad_enabled(True)
        loss *= one_sided_w

    return -loss.sum()


### Model Testing

In [ ]:
# load the model
interval = 20

net = NeuralNetwork()
net.load_state_dict(torch.load(os.path.join(os.getcwd(), f"ML_Model/CNN_1D_coordination_{interval}.pth")))

In [ ]:
# test number of feasible solutions
# test the model on the test set
net.eval()
net.to(device)

#### Testing of bit accuracy

In [ ]:
thres = 0.5

one_accuracy = []
zero_accuracy = []
bit_accuracy = []
running_loss = 0
mean_one = []
mean_zero = []

for j, data in enumerate(test_loader):
    
    net.eval()
    inputs_all, labels_all = data

    # do a for loop to perform perdiction for each scenario
    output_append = torch.tensor([], device=device)
    gt_append = torch.tensor([], device=device)

    for x in range(nbScen):
        
        inputs, labels = inputs_all[:,x,:,:].to(device), labels_all[:,x,:,].to(device)       
        optimizer.zero_grad()
        outputs = net(inputs)

        output_append = torch.concat((output_append, outputs),dim=1)
        gt_append = torch.concat((gt_append, labels),dim=1)

    output_append = output_append.reshape(-1,)
    gt_append = gt_append.reshape(-1,)

    loss_fn = nn.BCELoss()
    loss = loss_fn(output_append, gt_append)
    running_loss += loss.item()

    # start testing
    outputs_percent = output_append
    output_append = torch.where(output_append >= thres, torch.ceil(output_append), torch.floor(output_append)).reshape(-1,)
    # outputs = torch.round(outputs)

    one_labels = torch.where(gt_append == 1)
    zero_labels = torch.where(gt_append == 0)
    
    one_outputs = output_append[one_labels]
    zero_outputs = output_append[zero_labels]

    one_acc = 1 - torch.sum(torch.abs(1 - one_outputs)) / one_outputs.shape[0] # 1 minus percentage of error
    zero_acc = 1 - torch.sum(torch.abs(0 - zero_outputs)) / zero_outputs.shape[0]
    bit_acc = 1 - torch.sum(torch.abs(output_append - gt_append)) / gt_append.shape[0]

    one_accuracy.append(one_acc.cpu().detach().numpy())
    zero_accuracy.append(zero_acc.cpu().detach().numpy())
    bit_accuracy.append(bit_acc.cpu().detach().numpy())

    # mean acc
    id_1 = torch.where(output_append == 1)
    id_0 = torch.where(output_append == 0)

    p_1 = outputs_percent[id_1]
    p_0 = outputs_percent[id_0]


    y_1 = gt_append[id_1]
    y_0 = gt_append[id_0]

    y_1_1 = torch.where(y_1 == 1)
    y_1_0 = torch.where(y_1 == 0)
    y_0_1 = torch.where(y_0 == 1)
    y_0_0 = torch.where(y_0 == 0)

    avg_1 = torch.mean(torch.cat((p_1[y_1_1], torch.ones(y_1_0[0].shape[0]).to(device) - p_1[y_1_0])))
    avg_0 = torch.mean(torch.cat((p_0[y_0_1], torch.ones(y_0_0[0].shape[0]).to(device) - p_0[y_0_0])))

    # avg_1 = torch.mean(torch.cat((p_1[y_1_1],  p_1[y_1_0])))
    # avg_0 = torch.mean(torch.cat((p_0[y_0_1], p_0[y_0_0])))

    # avg_1 = torch.mean(p_1[y_1_1])
    # avg_0 = torch.mean(p_0[y_0_1])

    # avg_1 = torch.mean(p_1[y_1_0])
    # avg_0 = torch.mean(p_0[y_0_1])

    mean_one.append(avg_1.cpu().detach().numpy())
    mean_zero.append(avg_0.cpu().detach().numpy())

print("Average one bit accuracy", np.mean(one_accuracy))
print("Average zero bit accuracy", np.mean(zero_accuracy))
print("Average bit accuracy", np.mean(bit_accuracy))
print('Loss:', running_loss / len(test_loader))
print(np.mean(mean_one), np.mean(mean_zero))

### Test for baseline solving speed

In [ ]:
gurobi_env = gb.Env()
gurobi_env.setParam("OutputFlag", 0)
    
# loop through all test models and calculate average optimization time
opt_time = []
opt_val = []
opt_ones = []
for i, _ in enumerate(model_test):
    
    runtime = solTime_test[i]
    obj = objVal_test[i]
    
    print("Optimization time for model ", i, ": ", runtime)
    print("Optimization Value for model ", i, ": ", obj)
    
    opt_time.append(runtime)
    opt_val.append(obj)

    nbEV = np.max(schedule_test[i][:,0]) + 1
    binary_vars = y_test[i]

    opt_ones.append(np.count_nonzero(np.round(binary_vars)))


In [ ]:
opt_dict = {
    "opt_time": opt_time,
    "opt_val": opt_val,
    "opt_ones": opt_ones
}

result_path = os.path.join(os.getcwd(), f"Results")
with open(os.path.join(result_path, "opt_test.pkl"), 'wb') as f:
    pickle.dump(opt_dict, f)

In [ ]:
result_path = os.path.join(os.getcwd(), f"Results")
with open(os.path.join(result_path, "opt_test.pkl"), 'rb') as f:
    opt_dict = pickle.load(f)

opt_time = opt_dict["opt_time"]
opt_val = opt_dict["opt_val"]
opt_ones = opt_dict["opt_ones"]

In [ ]:
# flatten opt_time
print("Average optimization time: ", np.mean(opt_time))
opt_time_baseline = np.mean(opt_time)
# flatten opt_time
print("Average optimization value: ", np.mean(opt_val))
opt_val_baseline = np.mean(opt_val)
print("Number of ones: ", opt_ones)

In [ ]:
# read in paramaeters
data_dir = os.path.join(os.getcwd(), 'systemData')
EV_routes = pd.read_csv(os.path.join(data_dir, 'EV_routes.csv')).to_numpy()
bus_params = pd.read_csv(os.path.join(data_dir, 'bus_params.csv')).to_numpy()
# EV_schedules = pd.read_csv(os.path.join(data_dir, 'EV_schedules.csv')).to_numpy().astype("int32")

nbTime, nbBus, nbRoute = 48, bus_params.shape[0], EV_routes.shape[0]
traffic = np.zeros(nbTime-1)
traffic[14:20] = 1      # from 7-10am 
traffic[32:40] = 1      # from 4-8pm

charging_station = np.squeeze(pd.read_csv(os.path.join(data_dir, 'cs_params_variable.csv')).to_numpy())
non_charging_station = np.array([i for i in range(nbBus) if i not in charging_station])
nbCS = len(charging_station)

normal_nodes =  list(charging_station) + list(range(101,108))
virtual_nodes = list(range(201,205))
congest_nodes = list(range(301,324))

always_arc, normal_arc, congest_arc = [], [], []

for r in range(nbRoute):
    if (EV_routes[r,1] in (normal_nodes+virtual_nodes)) and (EV_routes[r,2] in (normal_nodes+virtual_nodes)) and (EV_routes[r,1] != EV_routes[r,2]):
        normal_arc.append(r)
    elif (EV_routes[r,1] in (normal_nodes+virtual_nodes)) and (EV_routes[r,2] in congest_nodes):
        congest_arc.append(r)
    else:
        always_arc.append(r)


In [ ]:
max_EV= 100
var_per_EV = (nbRoute*(nbTime-1) + nbCS*nbTime*2)

### Testing equality constraint with feasibility check

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)

gurobi_env = gb.Env()
gurobi_env.setParam("OutputFlag", 0)

# loop through all test models and calculate average optimization time
opt_time_feas = []
opt_val_feas = []
opt_ones_feas = []
opt_utilized_feas = []
for i, x in enumerate(model_test):
    # if i == 28:

    path = os.path.join(os.getcwd(), "dataGeneration/model_test")

    model = gb.read(os.path.join(path, f"coordination_{x}.mps"), env=gurobi_env)
    model.setParam("OutputFlag", 0)
    model.setParam("TimeLimit", 120*60)

    ################# timing the first code block #####################
    start = time.time()

    # deep learning prediciton
    output_append = torch.tensor([], device=device)
    gt_append = torch.tensor([], device=device)

    for s in range(nbScen):
        inputs = torch.tensor(np.expand_dims(test_dataset.X[i][s], axis=0), dtype=torch.float32) 
        inputs = inputs.to(device)    
        outputs = net(inputs) 

        # extract correct amount of binary before concat
        nbEV = np.max(schedule_test[i][:,0]) + 1
        outs = outputs[:,0:var_per_EV*nbEV]
        gts = torch.tensor(y_test[i,s,0:var_per_EV*nbEV], device=device).reshape(1,-1)

        output_append = torch.concat((output_append, outs),dim=1)
        gt_append = torch.concat((gt_append, gts),dim=1)

    output_append = output_append.reshape(-1,)
    gt_append = gt_append.reshape(-1,)

    ################## end #########################

    y_pred_binary =  (output_append).reshape(-1,).cpu().detach().numpy()
    # y_pred_binary = gt_append.reshape(-1,).cpu().detach().numpy()

    modelVars = model.getVars()

    # get correct amount of index
    bin_id = index_test[i][0:var_per_EV*nbEV*nbScen].reshape(-1,)

    one_threshold = (np.mean(mean_one))
    zero_threshold = (1-np.mean(mean_zero))

    # use equality constraint from here on
    for j in range(len(y_pred_binary)):
        if (y_pred_binary[j] >= one_threshold or y_pred_binary[j] <= zero_threshold):
            modelVars[bin_id[j]].setAttr("LB", round(y_pred_binary[j]))
            modelVars[bin_id[j]].setAttr("UB", round(y_pred_binary[j]))
            
    end = time.time()
    runtime1 = end - start

    opt_ones_feas.append(np.count_nonzero(y_pred_binary >= one_threshold))
    opt_utilized_feas.append(np.count_nonzero(y_pred_binary >= one_threshold)+  np.count_nonzero(y_pred_binary <= zero_threshold))

    ################# timing the second code block #####################
    start = time.time()

    model.optimize()

    end = time.time()
    runtime2 = end - start
    runtime = runtime1 + runtime2
    ################## end #########################
    
    try:
        print("Optimization time for model ", i, ": ", model.ObjVal)
        print("Optimization value for model ", i, ": ", runtime)
        opt_time_feas.append(runtime)
        opt_val_feas.append(model.ObjVal) 

    except:
        print("infeasible, reoptimising")

        model.dispose()

        ###### reset model and re-fix the values
        model = gb.read(os.path.join(path, f"coordination_{x}.mps"), env=gurobi_env)
        model.setParam("OutputFlag", 0)
        model.setParam("TimeLimit", 120*60)

        ################# timing the code block #####################
        start = time.time()

        modelVars = model.getVars()

        # get correct amount of index
        bin_id = index_test[i][0:var_per_EV*nbEV*nbScen].reshape(-1,)

        one_threshold = (np.mean(mean_one)) + 0.1
        zero_threshold = (1-np.mean(mean_zero))

        # use equality constraint from here on
        for j in range(len(y_pred_binary)):
            if (y_pred_binary[j] >= one_threshold or y_pred_binary[j] <= zero_threshold):
                modelVars[bin_id[j]].setAttr("LB", round(y_pred_binary[j]))
                modelVars[bin_id[j]].setAttr("UB", round(y_pred_binary[j]))
                
        end = time.time()
        runtime3 = end - start
        
        opt_ones_feas.append(np.count_nonzero(y_pred_binary >= one_threshold))
        opt_utilized_feas.append(np.count_nonzero(y_pred_binary >= one_threshold)+  np.count_nonzero(y_pred_binary <= zero_threshold))

        ################# timing the second code block #####################
        start = time.time()

        model.optimize()

        end = time.time()
        runtime4 = end - start
        runtime = runtime + runtime3 + runtime4 

        try:
            print("Optimization time for model ", i, ": ", model.ObjVal)
            print("Optimization value for model ", i, ": ", runtime)
            opt_time_feas.append(runtime)
            opt_val_feas.append(model.ObjVal) 
        except:
            print('infeasible')
            opt_time_feas.append(0)
            opt_val_feas.append(0) 

    
    model.dispose()

In [ ]:
opt_dict_feas = {
    "opt_time": opt_time_feas,
    "opt_val": opt_val_feas,
    "opt_ones": opt_ones_feas,
    "opt_utilized": opt_utilized_feas
}

result_path = os.path.join(os.getcwd(), f"Results")
with open(os.path.join(result_path, f"CNN_1D_{interval}_test_opt_feas.pkl"), 'wb') as f:
    pickle.dump(opt_dict_feas, f)

In [ ]:
with open(os.path.join(result_path, f"CNN_1D_{interval}_test_opt_feas.pkl"), 'rb') as f:
    opt_dict_feas = pickle.load(f)

opt_time_feas = opt_dict_feas["opt_time"]
opt_val_feas = opt_dict_feas["opt_val"]
opt_ones_feas = opt_dict_feas["opt_ones"]
opt_utilized_feas = opt_dict_feas["opt_utilized"]

In [ ]:
# flatten opt_time
opt_time_feas_mean = np.mean(opt_time_feas)
print("Average optimization time: ", np.mean(opt_time_feas))
# flatten opt_time
opt_val_feas_mean = np.mean(opt_val_feas)
print("Average optimization value: ", np.mean(opt_val_feas))
print("Number of feasible model", len(opt_time_feas))
print("Number of ones: ", opt_ones_feas)
print("Number of variable utilized ", opt_utilized_feas)

### Calculate time sped up by and optimality difference

In [ ]:
opt_time_feas_arr = np.array(opt_time_feas)
opt_val_feas_arr = np.array(opt_val_feas)
opt_time_arr = np.array(opt_time)
opt_val_arr = np.array(opt_val)

inf_id = np.where(opt_time_feas_arr == 0.0)

opt_time_feas_arr[inf_id] = opt_time_arr[inf_id]
opt_val_feas_arr[inf_id] = opt_val_arr[inf_id]

spd_up = 1-(np.array(opt_time_feas_arr)/np.array(opt_time_arr))
opt_loss = ((np.array(opt_val_feas_arr) - np.array(opt_val_arr))/np.array(opt_val_arr))

feasible_model_feas = len(opt_time_feas) / len(model_test) * 100
# ones_utilized_feas = np.array(opt_ones_feas) / np.array(opt_ones) *100
# prediction_utilized_feas = np.array(opt_utilized_feas) / len(index_test[0]) *100

print(f"Time sped up: {np.mean(spd_up)*100} %")
print(f"Optimality Loss: {np.mean(opt_loss)*100} %")
print(f"Feasible model: {feasible_model_feas} %")
